<a href="https://colab.research.google.com/github/microsoft/autogen/blob/main/notebook/agentchat_groupchat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Auto Generated Agent Chat: Group Chat

AutoGen offers conversable agents powered by LLM, tool or human, which can be used to perform tasks collectively via automated chat. This framework allows tool use and human participation through multi-agent conversation.
Please find documentation about this feature [here](https://microsoft.github.io/autogen/docs/Use-Cases/agent_chat).

This notebook is modified based on https://github.com/microsoft/FLAML/blob/4ea686af5c3e8ff24d9076a7a626c8b28ab5b1d7/notebook/autogen_multiagent_roleplay_chat.ipynb

## Requirements

AutoGen requires `Python>=3.8`. To run this notebook example, please install:
```bash
pip install pyautogen
```

In [ ]:
%%capture --no-stderr
# %pip install pyautogen~=0.1.0

In [ ]:
# ! pip install learn-to-pick

## Set your API Endpoint

The [`config_list_from_json`](https://microsoft.github.io/autogen/docs/reference/oai/openai_utils#config_list_from_json) function loads a list of configurations from an environment variable or a json file.

In [ ]:
# To capture the new agent during dev
import sys
sys.path.append("../")
import autogen

config_list_gpt4 = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    file_location="..",
    filter_dict={
        "model": ["gpt-4", "gpt-4-0314", "gpt4", "gpt-4-32k", "gpt-4-32k-0314", "gpt-4-32k-v0314"],
    },
)
# config_list_gpt35 = autogen.config_list_from_json(
#     "OAI_CONFIG_LIST",
#     filter_dict={
#         "model": {
#             "gpt-3.5-turbo",
#             "gpt-3.5-turbo-16k",
#             "gpt-3.5-turbo-0301",
#             "chatgpt-35-turbo-0301",
#             "gpt-35-turbo-v0301",
#         },
#     },
# )

It first looks for environment variable "OAI_CONFIG_LIST" which needs to be a valid json string. If that variable is not found, it then looks for a json file named "OAI_CONFIG_LIST". It filters the configs by models (you can filter by other keys as well). Only the gpt-4 models are kept in the list based on the filter condition.

The config list looks like the following:
```python
config_list = [
    {
        'model': 'gpt-4',
        'api_key': '<your OpenAI API key here>',
    },
    {
        'model': 'gpt-4',
        'api_key': '<your Azure OpenAI API key here>',
        'api_base': '<your Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2023-06-01-preview',
    },
    {
        'model': 'gpt-4-32k',
        'api_key': '<your Azure OpenAI API key here>',
        'api_base': '<your Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2023-06-01-preview',
    },
]
```

If you open this notebook in colab, you can upload your files by clicking the file icon on the left panel and then choose "upload file" icon.

You can set the value of config_list in other ways you prefer, e.g., loading from a YAML file.

## Construct Agents

In [ ]:
llm_config = {"config_list": config_list_gpt4, "seed": 42}
user_proxy = autogen.UserProxyAgent(
   name="User_proxy",
   system_message="A human admin.",
   code_execution_config={"last_n_messages": 2, "work_dir": "groupchat"},
   human_input_mode="TERMINATE"
)
coder = autogen.AssistantAgent(
    name="Coder",
    llm_config=llm_config,
)
pm = autogen.AssistantAgent(
    name="Product_manager",
    system_message="Creative in software product ideas.",
    llm_config=llm_config,
)
groupchat = autogen.GroupChat(agents=[user_proxy, coder, pm], messages=[], max_round=12)

In [ ]:
from autogen import oai

class autogen_llm_call:
    def __init__(self, config_list):
        self.config_list = config_list
    def predict(self, message):
        response = oai.ChatCompletion.create(
            config_list=self.config_list,
            prompt=message,
        )
        return oai.ChatCompletion.extract_text(response)[0]

autogen_llm_call(config_list=config_list_gpt4).predict("Hello, how are you?")

In [ ]:
import sys
sys.path.append("../")
from autogen.agentchat.contrib.learn_to_pick_agent import LearnToPickAgent
import learn_to_pick

# scoring_criteria_template = """Given message history: "{message_history}", the previously selected agent: "{last_speaker}" and the agents that could answer this message: "{agent}" rank how good or bad this agent selection is: "{picked}" """

ltp_config = {
    # "selection_scorer": learn_to_pick.AutoSelectionScorer(llm=autogen_llm_call(config_list=config_list_gpt4), scoring_criteria_template_str=scoring_criteria_template)
    "selection_scorer": None,
}

ltp_agent = LearnToPickAgent(
    llm_config=llm_config,
    learn_to_pick_config=ltp_config,)

In [ ]:
# hacky way to set the agent
groupchat.ltp_agent = ltp_agent
groupchat.previous_select_speaker_func = groupchat.select_speaker

In [ ]:
def custom_select_speaker(self, last_speaker: autogen.Agent, selector: autogen.ConversableAgent):
    
    # NOTE: Use with custom selection scorer set to AutoSelectionScorer (in config above)

    # The goal here is to demonstrate the usage of ltp agent in the role of the
    # deciding which agent needs to go next

    agent_desc = [{"name": agent.name, "desc": agent.system_message} for agent in self.agents]
    
    # TODO: Embed here should be exposed through LearnToPickAgent and not need to use
    # learn_to_pick directly
    msg = {
        "content": {
            "agent": learn_to_pick.Embed(LearnToPickAgent.ToSelectFrom(agent_desc)),
            "message": learn_to_pick.Embed(LearnToPickAgent.BasedOn(self.messages[-1]["content"])),
            "last_speaker": LearnToPickAgent.BasedOn(last_speaker.name),
            "message_history": self.messages,
        }
    }
    self.ltp_agent.receive(message=msg, sender=ltp_agent, request_reply=True, silent=True)
    reply = list(ltp_agent.chat_messages.values())[-1][-1]["content"]
    if "picked" in reply:
        agent_name = reply["picked"]["agent"]["name"]
        print(f"agent name picked: --{agent_name}--")
        try:
            return self.agent_by_name(str(agent_name))
        except ValueError as e:
            print(f"agent does not exist? got ValueError: {agent_name}, error: {e}")
            return self.next_agent(last_speaker, agents)
    else:
        print(f"no agent picked, problem with calling ltp agent?")
        return self.next_agent(last_speaker, agents)

In [ ]:
def custom_select_speaker_warm_start(self, last_speaker: autogen.Agent, selector: autogen.ConversableAgent):
    
    # NOTE: Use with selection_scorer set to None in the config
    # The goal here is to demonstrate that the ltp agent is learning "warm start" from what
    # AutoGen GroupChat manager is doing
    # Can save progress and resume (not demonstrated below, not sure I exposed that in the agent yet)

    # What did the group manager pick?
    group_chat_agent_selection = self.previous_select_speaker_func(last_speaker, selector)
    
    agent_desc = [{"name": agent.name, "desc": agent.system_message} for agent in self.agents]
    msg = {
        "content": {
            "agent": learn_to_pick.Embed(LearnToPickAgent.ToSelectFrom(agent_desc)),
            "message": learn_to_pick.Embed(LearnToPickAgent.BasedOn(self.messages[-1]["content"])),
            "last_speaker": LearnToPickAgent.BasedOn(last_speaker.name),
            "message_history": self.messages,
        }
    }
    self.ltp_agent.receive(message=msg, sender=ltp_agent, request_reply=True, silent=True)
    ltp_reply = list(ltp_agent.chat_messages.values())[-1][-1]["content"]
    if "picked" in ltp_reply:
        ltp_agent_name = ltp_reply["picked"]["agent"]["name"]
        print(f"agent name picked by ltp agent: --{ltp_agent_name}--")
        
        if group_chat_agent_selection.name == ltp_agent_name:
            ltp_agent.learn_from_user_feedback(1.0, ltp_reply)
        else:
            # -1 or 0.0 here?
            ltp_agent.learn_from_user_feedback(0.0, ltp_reply)
    
    return group_chat_agent_selection

In [ ]:
# groupchat.select_speaker = custom_select_speaker.__get__(groupchat)
groupchat.select_speaker = custom_select_speaker_warm_start.__get__(groupchat)

In [ ]:
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

## Start Chat

In [ ]:
user_proxy.initiate_chat(manager, message="Find a latest paper about gpt-4 on arxiv and find its potential applications in software.")
# type exit to terminate the chat